<a href="https://colab.research.google.com/github/sapt36/Google_Map_Crawler/blob/main/GoogleMapCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安裝必要的套件
!pip install -U googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=ee78adec5d2ee25093c41dda2f8db29254b7eeb77dbffbeadd8e15d882fb800e
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
import pandas as pd
import json
import googlemaps
from geopy.distance import geodesic
import time

# 初始化 Google Maps API 客戶端
api_key = ""  # 您的 Google Maps API 金鑰
gmaps = googlemaps.Client(key=api_key)

# 定義函數來獲取附近的景點資訊（支援多頁，因為google api 上限20，需要多頁才能突破上限到如下的 max_results=100）
def get_nearby_spots(api_key, location, radius, keyword, max_results=100):  # max_results 參數，代表最多出現的結果數(可增加)
    spots = []
    gmaps = googlemaps.Client(key=api_key)
    next_page_token = None

    # 繼續請求直到獲得所需數量的景點
    while len(spots) < max_results:
        # 使用 places_nearby API 查找附近的景點，檢查是否需要使用 page_token
        try:
            if next_page_token:
                # sleep 等待 token 有效，避免 INVALID_REQUEST
                time.sleep(2)
                spots_result = gmaps.places_nearby(
                    location=location,
                    radius=radius,
                    keyword=keyword,
                    page_token=next_page_token
                )
            else:
                spots_result = gmaps.places_nearby(
                    location=location,
                    radius=radius,
                    keyword=keyword
                )

            # 遍歷每個景點並獲取基本資訊
            for spot in spots_result['results']:
                place_id = spot['place_id']
                name = spot['name']
                address = spot['vicinity']
                rating = spot.get('rating', 'N/A')
                lat = spot['geometry']['location']['lat']
                lng = spot['geometry']['location']['lng']
                distance = geodesic((float(location.split(",")[0]), float(location.split(",")[1])), (lat, lng)).meters

                # 使用 place API 查詢詳細資訊
                details = gmaps.place(place_id=place_id, fields=["review", "formatted_phone_number", "website"])
                reviews = []

                # 提取熱門評論
                if "reviews" in details["result"]:
                    for review in details["result"]["reviews"]:
                        reviews.append({
                            "author_name": review["author_name"],
                            "rating": review["rating"],
                            "text": review["text"],
                            "time": review["relative_time_description"]
                        })

                # 提取電話和網址
                phone_number = details["result"].get("formatted_phone_number", "N/A")
                website = details["result"].get("website", "N/A")

                # 將景點資訊和評論加入列表
                spots.append({
                    "名稱": name,
                    "地址": address,
                    "評分": rating,
                    "電話": phone_number,
                    "網址": website,
                    "距離": distance,
                    "熱門評論": reviews
                })

                # 停止迴圈當取得到足夠數量的景點
                if len(spots) >= max_results:
                    break

            # 設定下一頁 token（若有）
            next_page_token = spots_result.get("next_page_token", None)

            if not next_page_token:
                break  # 沒有更多結果則停止

        except googlemaps.exceptions.ApiError as e:
            print(f"API Error: {e}")
            break
        except googlemaps.exceptions.Timeout:
            print("Request Timeout. Retrying...")
            time.sleep(2)
            continue
        except googlemaps.exceptions.TransportError as e:
            print(f"Transport Error: {e}")
            break
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            break

    # 根據距離排序並編號
    spots.sort(key=lambda x: x["距離"])
    for idx, spot in enumerate(spots, start=1):
        spot["編號"] = idx

    return spots

# 設定位置座標、範圍和關鍵字
location = "25.03640955354205, 121.52005195201416" # 座標位於：國立中正紀念堂
radius = 50000  # 擴大範圍可獲取更多景點 50000 = 50KM
keyword = "spot"  # 可以根據需求更改關鍵字

# 獲取附近景點資訊，包含熱門評論、電話和網址
spots_data = get_nearby_spots(api_key, location, radius, keyword)

# 將結果轉為 JSON 格式並儲存檔案
with open("sorted_spots_data_with_reviews_contact.json", "w", encoding="utf-8") as f:
    json.dump(spots_data, f, ensure_ascii=False, indent=4)

print("已輸出 sorted_spots_data_with_reviews_contact.json 檔案")


已輸出 sorted_spots_data_with_reviews_contact.json 檔案
